![iamge](./test2/tool_calling.png)

In [21]:
from dotenv import load_dotenv

# for WOLFRAM_ALPHA_KEY and TAVILY_API_KEY
load_dotenv()
from utils import llama31
from utils import cprint
import json

In [22]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

23 November 2024


In [23]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [24]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [25]:
response = llama31(prompt)
print(response)



brave_search.call(query="Menlo Park, California weather")


In [26]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [27]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of my cut-off knowledge date of 01 March 2023, the estimated population of California is approximately 39.5 million people, according to the United States Census Bureau. However, please note that this information may have changed since my cut-off date. For the most up-to-date information, I recommend checking the latest data from the United States Census Bureau or other reliable sources.


#### tavily web search

In [28]:
from tavily import TavilyClient
tavily_client = TavilyClient()

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732297262, 'localtime': '2024-11-22 09:41'}, 'current': {'last_updated_epoch': 1732296600, 'last_updated': '2024-11-22 09:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.1, 'wind_kph': 16.2, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 100, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 12.

### Reprompting Llama with search tool response

In [29]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732297262, 'localtime': '2024-11-22 09:41'}, 'current': {'last_updated_epoch': 1732296600, 'last_updated': '2024-11-22 09:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.1, 'wind_kph': 16.2, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 100, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 12.2, 'windchill_f': 54.0, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 11.9, 'dewpoint_f': 53.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.1, 'gust_mph': 14.8, 'gust_kph': 23.8}}


In [30]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 November 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>

brave_search.call(query="Menlo Park, California weather")<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732297262, 'localtime': '2024-11-22 09:41'}, 'current': {'last_updated_epoch': 1732296600, 'last_updated': '2024-11-22 09:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.1, 'wind_kph': 16.2, 'wind_degree': 156, 'wi

In [31]:
response = llama31(prompt)
print(response)

The current weather in Menlo Park, California is overcast with a temperature of 17.2°C (63.0°F) and a wind speed of 10.1 mph (16.2 kph) from the SSE. The humidity is 72% and the pressure is 1015 mb. There is no precipitation.


### Change to higher-level message

In [32]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]

In [33]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


In [37]:
response

'The current weather in Menlo Park, California is overcast with a temperature of 17.2°C (63.0°F) and a wind speed of 10.1 mph (16.2 kph) from the SSE. The humidity is 72% and the pressure is 1015.0 mb. There is no precipitation.'

In [38]:
search_result

"{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732297262, 'localtime': '2024-11-22 09:41'}, 'current': {'last_updated_epoch': 1732296600, 'last_updated': '2024-11-22 09:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.1, 'wind_kph': 16.2, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 100, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 12.2, 'windchill_f': 54.0, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 11.9, 'dewpoint_f': 53.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.1, 'gust_mph': 14.8, 'gust_kph': 23.8}}"

In [34]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Menlo Park, California?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [35]:
response = llama31(messages)
print(response)

The current weather in Menlo Park, California is overcast with a temperature of 17.2°C (63.0°F) and a wind speed of 10.1 mph (16.2 kph) from the SSE. The humidity is 72% and the pressure is 1015.0 mb. There is no precipitation.


## The Wolfram Alpha tool

In [36]:

from wolframalpha import Client

def wolfram_alpha(query: str) -> str:
   
    client = Client()
    result = client.query(query)

    results = []
    for pod in result.pods:
        if pod["@title"] == "Result" or pod["@title"] == "Results":
          for sub in pod.subpods:
            results.append(sub.plaintext)

    return '\n'.join(results)

In [3]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0? "

In [10]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


In [11]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


In [12]:
#### check result  calling it
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


In [13]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [14]:
response = llama31(messages)
print(response)

The solutions to the equation x^3 - 2x^2 - x + 2 = 0 are x = ±1 and x = 2.


### The code interpreter built-in too

In [15]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [17]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]
response = llama31(messages)
print(response)

import math

# Define the variables
loan_amount = 1000000 - (1000000 * 0.2)  # $1M with 20% down payment
annual_interest_rate = 0.06  # 6% interest rate
years = 30  # 30-year mortgage
months = years * 12  # Convert years to months

# Calculate the monthly interest rate
monthly_interest_rate = annual_interest_rate / 12

# Calculate the monthly payment
monthly_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate) ** months) / ((1 + monthly_interest_rate) ** months - 1)

# Calculate the total payment
total_payment = monthly_payment * months

# Calculate the total interest paid
total_interest_paid = total_payment - loan_amount

print(f"Monthly payment: ${monthly_payment:.2f}")
print(f"Total payment: ${total_payment:.2f}")
print(f"Total interest paid: ${total_interest_paid:.2f}")
